# 06_Analysis_Relapse_and_2nd_transplant_survival

# Relapse & Second Transplant vs. Survival of Patients

Dataset: *Bone Marrow Transplantation: Children*

*Exporatory Anlysis. Not included in the slides.*

We want to look at the relationship between relapse and secondary
transplants in patients and how it affects their odds of survival.

To do this, we look at how many relapsed, needed a second transplant,
and how many of them survived and when they where observed. This is done
using X variables taken from the cleaned augmented dataset, described in
table 1, and then performing an analysis of the relationship between
them.

| Variable | Description | Value |
|:--------:|:-----------:|:-----:|

`Txpostrelapse` \| Second Bone Marrow transplantation after relapse \|
Yes, No \|  
`Relapse` \| Reoccurance of disease \| Yes, No \|  
`survival_time` \| Time of Observation (Alive) or time of event in days
(Dead) \| Yes, No \|  
`survival_status` \| Survival status \| Yes, No \|  
`Diseasegroup` \| Type of disease \| Malignant, Nonmalignant \|

: Table 1: The data used for this analysis is from `03_dat_aug` using
these variables:`Txpostrelapse`, `Relapse`, `survival_time`,
`survival_status` & `Diseasegroup`.

## Data Wrangling

### Initiate Libraries

We start by loading the relevant libraries.

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   4.0.0     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

here() starts at /net/pupil1/home/people/s150251/projects/group_24_project

We then load a subset of the cleaned and augmented data frame as a
tibble.

In [ ]:
bm_data <- read_tsv(file = here("data/03_dat_aug.tsv.gz"),
                            show_col_types = FALSE)

It contains 186 values and follows the guidelines for tidy data, with:

-   Each variable having its own column.

-   Each observation having its own row.

-   Each value having its own cell.

And all values used in this analysis does not contain missing values,
see table 1 for variables.

The full tibble can be seen here.

In [ ]:
bm_data

# A tibble: 186 × 35
   Recipientgender Stemcellsource Donorage Donorage35 IIIV  Gendermatch DonorABO
   <chr>           <chr>             <dbl>      <dbl> <chr> <chr>       <chr>   
 1 Male            Bone marrow        23.3          0 Yes   Other       B       
 2 Male            Bone marrow        26.4          0 Yes   Other       B       
 3 Female          Bone marrow        39.7          1 Yes   Other       A       
 4 Female          Pheripheral b…     33.4          0 No    Other       A       
 5 Male            Bone marrow        27.4          0 No    Other       AB      
 6 Female          Pheripheral b…     34.5          0 Yes   Other       0       
 7 Male            Bone marrow        21.4          0 Yes   Other       0       
 8 Male            Pheripheral b…     32.6          0 No    Other       AB      
 9 Male            Pheripheral b…     28.8          0 Yes   Female to … A       
10 Female          Pheripheral b…     29.7          0 No    Other       0       
# ℹ 176

## Data Analysis

We start by generating the subset for the first part of the analysis
`relapse_surv_data` containing 4 variables.

In [ ]:
relapse_surv_data <- bm_data |>
  select(Txpostrelapse,
         Relapse,
         survival_status,
         survival_time)

relapse_surv_data

# A tibble: 186 × 4
   Txpostrelapse Relapse survival_status survival_time
   <chr>         <chr>   <chr>                   <dbl>
 1 No            Yes     Dead                      163
 2 No            Yes     Dead                      435
 3 No            No      Dead                       53
 4 No            No      Alive                    2043
 5 Yes           No      Alive                    2800
 6 No            No      Dead                       41
 7 No            No      Dead                       45
 8 No            No      Alive                     671
 9 No            No      Alive                     676
10 No            No      Alive                     599
# ℹ 176 more rows

### Survival

We want to look at the relationship between survival of patients and the
time of observation or event. This is done by visualizing
`survival_time` and `survival_status` from subset: `relapse_surv_data`.

In [ ]:
#| label: Boxplot of time of survival stradified by survival status

relapse_surv_data |>
  ggplot(data = _,
         mapping = aes(x = survival_status,
                       y = survival_time,
                       fill = survival_status)) +
  geom_boxplot(alpha = 0.5) + 
  labs(x = "Survival Status",
       y = "Survival Time [days]",
       title = "Time differences in days between good and bad outcomes for Patients",
       subtitle = "Time stamps are either Observations for living Patients and Events for dead Patients",
       caption = "Data from the Bone Marrow Transplantation: Children dataset") +
  theme_minimal(base_family = "Avenir", 
                base_size = 12) +
  theme(legend.position = "Bottom")

We can see that Patients with bad outcomes die within ADD STATISTICS
days and observations taken from patients with good outcomes usually
happen around ADD STATISTICS day. This says something about the
observation time needed to see if an rejection of the transplantation
happens.

### Relapse in Relation to Survival

Next we will look at the rate of relapses and how it relates to survival
status of patients.

In [ ]:
relapse_surv_data |>
  count(survival_status,Relapse) |>
  ggplot(mapping = aes(x = Relapse,
                       y = n,
                       fill = survival_status)) +
  geom_col(position = position_dodge(
    preserve = "single"),
    colour = "black",
    alpha = 0.5) +
  geom_hline(yintercept = 0) +
  theme_minimal(base_size = 12) +
  theme(legend.position = "bottom",
        panel.grid.major.x = element_blank())

We see a large number of the patents that survived did not relapse. This
is shown in numbers below.

In [ ]:
relapse_surv_data |>
  count(Relapse,survival_status)

# A tibble: 4 × 3
  Relapse survival_status     n
  <chr>   <chr>           <int>
1 No      Alive              96
2 No      Dead               62
3 Yes     Alive               5
4 Yes     Dead               23

We see that the majority of relapsed patients did not survive. This is
something we will look more into by looking who got a second transplant.
INSERT STASTISTICS IN THIS SECTION!

### Secondary transplant after relapse in relation to survival

First we look at who got a second transplant. NEED BETTER TABLE!

In [ ]:
relapse_surv_data |>
  group_by(Txpostrelapse) |>
  count(Relapse,survival_status)

# A tibble: 7 × 4
# Groups:   Txpostrelapse [2]
  Txpostrelapse Relapse survival_status     n
  <chr>         <chr>   <chr>           <int>
1 No            No      Alive              87
2 No            No      Dead               54
3 No            Yes     Alive               5
4 No            Yes     Dead               17
5 Yes           No      Alive               9
6 Yes           No      Dead                8
7 Yes           Yes     Dead                6

We then create a new subset with a new variable: `tx_history`, based on:
`Txpostrelapse` & `Relapse`.

In [ ]:
transplant_surv_data <- relapse_surv_data |>
  mutate(
    tx_history = case_when(
      Txpostrelapse == "Yes" & Relapse == "Yes" ~  "2nd Transplant & Relapse",
      Txpostrelapse == "Yes" & Relapse == "No" ~  "2nd Transplant & no Relapse*",
      Txpostrelapse == "No" & Relapse == "Yes" ~  "No Transplant & Relapse",
      Txpostrelapse == "No" & Relapse == "No" ~  "No Transplant & no Relapse"))

Visualization of 2nd Transplant and relapse data.

In [ ]:
transplant_surv_data |>
  count(tx_history,survival_status) |>
  ggplot(mapping = aes(x = tx_history,
                       y = n,
                       fill = factor(survival_status))) +
  geom_col(position = position_dodge(
    preserve = "single"),
    colour = "black",
    alpha = 0.5) +
  geom_hline(yintercept = 0) +
  theme_minimal(base_size = 12) +
  labs(caption = "*No Relapse means no remission before 2nd transplant",
       fill = "Survival Status",
       x = "Transplant and relapse History") +
  theme(legend.position = "bottom",
        panel.grid.major.x = element_blank(),
        axis.text.x = element_text(angle = 10,
                                   hjust = 0.5,
                                   vjust = 1))

We see a difference in the number of surviving patients when we look at
the number of patients receiving a second transplantation, with few
surviving after having a relapse.

Patients, who need a second transplantation and still have not gone into
remission (no relapse) have a 50/50-ish chance of survival.

Patients with relapse have bad odds of surviving.

Patients, who does not need a second transplant have a better outcome
with surviving.

**Outcome:** 2nd transplant seem to help some after relapse, but odds
are grim in case of relapse. Can’t say anything definitivly.

### Look at ‘Diseasegroup’ in relation to relapse and survival

`Diseasegroup` describes if the type of disease is Malignant (1) or
Nonmalignant (0)

We create a new subset with `Diseasegroup`. *2nd transplant is kept in
for now, might remove later.*

In [ ]:
malign_surv_data <- bm_data |>
  select(Txpostrelapse,
         Relapse,
         Diseasegroup,
         survival_status,
         survival_time)

Then we add a variable `type_history`, which describes the type of
disease the patient had.

In [ ]:
malign_surv_data <- malign_surv_data |>
  mutate(
    type_history = case_when(
      Diseasegroup == "Malignant" & Relapse == "Yes" ~  "Malignant & Relapse",
      Diseasegroup == "Malignant" & Relapse == "No" ~  "Malignant & no Relapse*",
      Diseasegroup == "Nonmalignant" & Relapse == "Yes" ~  "Nonmalignant & Relapse",
      Diseasegroup == "Nonmalignant" & Relapse == "No" ~  "Nonmalignant & no Relapse"))

Next we see if it affects relapse and survival.

In [ ]:
malign_surv_data |>
  count(type_history,survival_status) |>
  ggplot(mapping = aes(x = type_history,
                       y = n,
                       fill = factor(survival_status))) +
  geom_col(position = position_dodge(
    preserve = "single"),
    colour = "black",
    alpha = 0.5) +
  geom_hline(yintercept = 0) +
  theme_minimal(base_size = 12) +
  labs(caption = "*No Relapse means no remission",
       fill = "Survival Status",
       x = "Disease type and relapse status for Patients") +
  theme(legend.position = "bottom",
        panel.grid.major.x = element_blank(),
        axis.text.x = element_text(angle = 10,
                                   hjust = 0.5,
                                   vjust = 1))

Possible next clues:

-   Look at ‘Disease’? or ‘Riskgroup’?